In [1]:
# предобработка данных
# отключаем предупреждения Anaconda
import warnings
warnings.simplefilter('ignore')

import pandas as pd
import numpy as np

from datetime import datetime

dateparse = lambda x: pd.to_datetime(x)
gl = pd.read_csv('Crime_Data_from_2010.csv',
                                 parse_dates=['Date Reported','Date Occurred'],
                                 date_parser=dateparse,
                                 infer_datetime_format=True)

gl_float = gl.select_dtypes(include=['float'])
converted_float = gl_float.apply(pd.to_numeric,downcast='float')
gl_int = gl.select_dtypes(include=['int'])
converted_int = gl_int.apply(pd.to_numeric,downcast='unsigned')

gl[converted_int.columns] = converted_int
gl[converted_float.columns] = converted_float
# заменяем исходные числовые столбцы
# оптимизированными
gl[converted_int.columns] = converted_int
gl[converted_float.columns] = converted_float

gl['Area ID']= gl['Area ID'].astype('uint16')
gl['Victim Age'].fillna(value = 0, inplace=True)
gl['Victim Age']= gl['Victim Age'].astype('uint8')


categ = ['Premise Description','Status Code','Status Description','Victim Descent','Victim Sex'
         ,'Crime Code Description','Area ID']
for category in categ:
    gl[category] = gl[category].astype('category')

def good_time(x):
    if x<10:
        x*=100
    return x
gl['Time Occurred'] = gl['Time Occurred'].apply(good_time)

gl['Full_Time_Occurred'] = pd.to_datetime(gl['Time Occurred'].astype(str)+gl['Date Occurred'].astype(str),
                                          format='%H%M%Y-%m-%d')


gl['Time Occurred'] = pd.to_datetime(gl['Time Occurred'], format='%H%M')

codes = ['Crime Code 1','Crime Code 2','Crime Code 3','Crime Code 4','Premise Code','Weapon Used Code','Reporting District']
for code in codes:
    gl[code] = gl[code].fillna(value = 0).astype('uint16')

gl['Weekday'] = gl['Date Occurred'].dt.strftime("%A").astype('category')

gl.info(memory_usage='deep')

gl.to_csv('LA_crime_compressed.csv')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1692056 entries, 0 to 1692055
Data columns (total 28 columns):
 #   Column                  Dtype         
---  ------                  -----         
 0   DR Number               int64         
 1   Date Reported           datetime64[ns]
 2   Date Occurred           datetime64[ns]
 3   Time Occurred           datetime64[ns]
 4   Area ID                 category      
 5   Area Name               object        
 6   Reporting District      uint16        
 7   Crime Code              int64         
 8   Crime Code Description  category      
 9   MO Codes                object        
 10  Victim Age              uint8         
 11  Victim Sex              category      
 12  Victim Descent          category      
 13  Premise Code            uint16        
 14  Premise Description     category      
 15  Weapon Used Code        uint16        
 16  Weapon Description      object        
 17  Status Code             category      
 18  St

сохраним типы данных

In [31]:
import json
res = gl.dtypes.to_frame('dtypes').reset_index()
# first create dictionary
d = res.set_index('index')['dtypes'].astype(str).to_dict()

with open('types.json', 'w') as f:
    json.dump(d, f)